Pythonでpdfデータにあるテーブルデータを一括でcsvに直す方法 - Qiita  
https://qiita.com/risako_/items/0c625a6bcb1cd80cf259  

tabula_example.ipynb - Colaboratory  
https://colab.research.google.com/github/chezou/tabula-py/blob/master/examples/tabula_example.ipynb#scrollTo=q6FGPenCluQz


----

#Setup

In [178]:
!pip install -q tabula-py

In [179]:
import tabula
import pandas as pd

# Main

In [180]:
# 対象のpdfはlattice=Trueにしないとデータ欠損が起きる
target_year = 2019
pdf_path = 'http://www.eiren.org/toukei/img/eiren_kosyu/data_' + str(target_year) + '.pdf'
df = tabula.read_pdf(pdf_path, lattice=True)[0]

print(df.columns)

# Rename columns
df = df.rename(columns={'順位': 'rank', '公開月': 'release_date', '作      品      名': 'title', '興収(単位:億円)': 'box_office', '配給会社': 'agency'})
df.head()

'pages' argument isn't specified.Will extract only from page 1 by default.
Got stderr: Mar 02, 2021 1:59:00 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Mar 02, 2021 1:59:00 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14
Mar 02, 2021 1:59:00 AM org.apache.fontbox.ttf.CmapSubtable processSubtype14



Index(['順位', '公開月', '作      品      名', '興収(単位:億円)', '配給会社'], dtype='object')


,rank,release_date,title,box_office,agency
0,1,7月,天気の子,141.9,東宝
1,2,4月,名探偵コナン 紺青の拳(フィスト),93.7,東宝
2,3,4月,キングダム,57.3,東宝/SPE
3,4,8月,劇場版「ONE PIECE STAMPEDE」,55.5,東映
4,5,3月,映画ドラえもん のび太の月面探査記,50.2,東宝


In [181]:
df.set_index('rank', inplace=True)
df.head()

,release_date,title,box_office,agency
rank,,,,
1,7月,天気の子,141.9,東宝
2,4月,名探偵コナン 紺青の拳(フィスト),93.7,東宝
3,4月,キングダム,57.3,東宝/SPE
4,8月,劇場版「ONE PIECE STAMPEDE」,55.5,東映
5,3月,映画ドラえもん のび太の月面探査記,50.2,東宝


In [182]:
# 興行収入を決算資料などと合わせやすいように百万円の単位に変換
df['box_office'] = df['box_office'].apply(lambda x: x * 100).apply(int)
df.head()

,release_date,title,box_office,agency
rank,,,,
1,7月,天気の子,14190,東宝
2,4月,名探偵コナン 紺青の拳(フィスト),9370,東宝
3,4月,キングダム,5730,東宝/SPE
4,8月,劇場版「ONE PIECE STAMPEDE」,5550,東映
5,3月,映画ドラえもん のび太の月面探査記,5020,東宝


In [183]:
df['release_year'] = list(pd.Series(df.index).apply(lambda x: target_year))
df['release_month'] = list(pd.Series(df.index).apply(lambda x: 0))

for index, row in df.iterrows():
  date = row['release_date'].split('/')

  if len(date) > 1:
    df.iat[index-1, 4] = int(date[0]) + 2000

  df.iat[index-1, 5] = date[-1].replace('月', '')

df.drop(columns=['release_date'], inplace=True)

In [184]:
df = df.reindex(columns=['release_year', 'release_month', 'title', 'box_office', 'agency'])
df.head()

,release_year,release_month,title,box_office,agency
rank,,,,,
1,2019,7,天気の子,14190,東宝
2,2019,4,名探偵コナン 紺青の拳(フィスト),9370,東宝
3,2019,4,キングダム,5730,東宝/SPE
4,2019,8,劇場版「ONE PIECE STAMPEDE」,5550,東映
5,2019,3,映画ドラえもん のび太の月面探査記,5020,東宝
